In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/london-homes-energy-data/london_energy.csv
/kaggle/input/london-weather-data/london_weather.csv


# London Household Energy Consumption

EDA practice notebook, found 2 interesting data sets which I'll try to merge and explore

In [2]:
df_energy = pd.read_csv('../input/london-homes-energy-data/london_energy.csv')

print(df_energy.shape)
df_energy.head()


(3510433, 3)


,LCLid,Date,KWH
0,MAC000002,2012-10-12,7.098
1,MAC000002,2012-10-13,11.087
2,MAC000002,2012-10-14,13.223
3,MAC000002,2012-10-15,10.257
4,MAC000002,2012-10-16,9.769


In [3]:
df_weather = pd.read_csv('../input/london-weather-data/london_weather.csv')

print(df_weather.shape)
df_weather.head()

(15341, 10)


,date,cloud_cover,sunshine,global_radiation,max_temp,mean_temp,min_temp,precipitation,pressure,snow_depth
0,19790101,2.0,7.0,52.0,2.3,-4.1,-7.5,0.4,101900.0,9.0
1,19790102,6.0,1.7,27.0,1.6,-2.6,-7.5,0.0,102530.0,8.0
2,19790103,5.0,0.0,13.0,1.3,-2.8,-7.2,0.0,102050.0,4.0
3,19790104,8.0,0.0,13.0,-0.3,-2.6,-6.5,0.0,100840.0,2.0
4,19790105,6.0,2.0,29.0,5.6,-0.8,-1.4,0.0,102250.0,1.0


## Homogenise date features

In [4]:
# Check date columns dtype

df_weather['date'].dtypes, df_energy['Date'].dtypes

(dtype('int64'), dtype('O'))

🔗 To convert YYYY-MM-DD to YYYYMMDD [here](https://stackoverflow.com/questions/43133605/convert-integer-yyyymmdd-to-date-format-mm-dd-yyyy-in-python) and [here](https://stackoverflow.com/questions/9750330/how-to-convert-integer-into-date-object-python)

🔗 To convert YYYYMMDD to YYYY-MM-DD [here](https://stackoverflow.com/questions/46834732/convert-pandas-datetime-column-yyyy-mm-dd-to-yyyymmdd)

In [5]:
# Rename df_energy date column to be the same as in df_weather

df_energy.rename(columns={'Date': 'date'}, inplace=True)

df_energy.columns

Index(['LCLid', 'date', 'KWH'], dtype='object')

### Convert date column to pandas datetime object

In [6]:
df_energy['date'] = pd.to_datetime(df_energy['date'].astype('string'))

df_energy.head()

,LCLid,date,KWH
0,MAC000002,2012-10-12,7.098
1,MAC000002,2012-10-13,11.087
2,MAC000002,2012-10-14,13.223
3,MAC000002,2012-10-15,10.257
4,MAC000002,2012-10-16,9.769


### Convert date column to pandas datetime object

In [7]:
df_weather['date'] = pd.to_datetime(df_weather['date'].astype('string'))

df_weather.head()

,date,cloud_cover,sunshine,global_radiation,max_temp,mean_temp,min_temp,precipitation,pressure,snow_depth
0,1979-01-01,2.0,7.0,52.0,2.3,-4.1,-7.5,0.4,101900.0,9.0
1,1979-01-02,6.0,1.7,27.0,1.6,-2.6,-7.5,0.0,102530.0,8.0
2,1979-01-03,5.0,0.0,13.0,1.3,-2.8,-7.2,0.0,102050.0,4.0
3,1979-01-04,8.0,0.0,13.0,-0.3,-2.6,-6.5,0.0,100840.0,2.0
4,1979-01-05,6.0,2.0,29.0,5.6,-0.8,-1.4,0.0,102250.0,1.0


🔗 Remove rows before a date [here](https://stackoverflow.com/questions/51065979/delete-rows-with-dates-before-the-required-date-point-based-on-key-value)

### Combine energy and weather dataframes

In [8]:
df_energy['date'].astype('datetime64[ns]')
df_weather['date'].astype('datetime64[ns]')

df = df_energy.merge(df_weather, on='date')
df.head()

,LCLid,date,KWH,cloud_cover,sunshine,global_radiation,max_temp,mean_temp,min_temp,precipitation,pressure,snow_depth
0,MAC000002,2012-10-12,7.098,5.0,6.4,106.0,13.5,12.3,9.7,0.0,100580.0,0.0
1,MAC000003,2012-10-12,13.434,5.0,6.4,106.0,13.5,12.3,9.7,0.0,100580.0,0.0
2,MAC000004,2012-10-12,1.522,5.0,6.4,106.0,13.5,12.3,9.7,0.0,100580.0,0.0
3,MAC000005,2012-10-12,4.448,5.0,6.4,106.0,13.5,12.3,9.7,0.0,100580.0,0.0
4,MAC000006,2012-10-12,4.074,5.0,6.4,106.0,13.5,12.3,9.7,0.0,100580.0,0.0


In [9]:
df.shape

(3510433, 12)

In [10]:
# Check earliest and latest dates

earliest = min(df['date'])
latest = max(df['date'])

earliest, latest

(Timestamp('2011-11-23 00:00:00'), Timestamp('2014-02-28 00:00:00'))